In [1]:
import scipy
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
heart = pd.read_excel("/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Summary_Pub_Deidentified.xlsx")
df = heart.copy().fillna(0)

In [3]:
df.drop(["Mitral valve Peak velocity (m/sec)",
         "Mitral valve Mean gradient(mmHg)", 
         "Echo available",
        "Date of collection",
        "Date of echo"], 
        inplace = True, axis = 1)
df.drop(df.tail(1).index,inplace=True)

In [4]:
idx = np.where((df['History of MI'] == 0.0)  & (df['History of CABG'] == 0.0)  & (df['History of PCI'] == 0.0))

In [5]:
healty = df.loc[idx]

In [6]:
diseased = df.drop(idx[0], axis = 0)

In [7]:
from scipy.stats import mannwhitneyu

The default assumption or null hypothesis is that there is no difference between the distributions of the data samples. Rejection of this hypothesis suggests that there is likely some difference between the samples. More specifically, the test determines whether it is equally likely that any randomly selected observation from one sample will be greater or less than a sample in the other distribution. If violated, it suggests differing distributions.

Fail to Reject H0: Sample distributions are equal.
Reject H0: Sample distributions are not equal.
For the test to be effective, it requires at least 20 observations in each data sample.



In [8]:
record = []

In [9]:
for i in range(10):
    random_healty = healty.sample(n = 20, axis = 0).drop(["Patient ID", "Start", "Duration"], axis = 1)
    random_healty = pd.get_dummies(random_healty)


    random_diseased = diseased.sample(n = 20, axis = 0).drop(["Patient ID", "Start", "Duration"], axis = 1)
    random_diseased = pd.get_dummies(random_diseased)
    
    
    record.append(mannwhitneyu(random_healty, random_diseased, use_continuity=False, alternative = 'two-sided'))

In [10]:
columns = diseased.copy().drop(["Patient ID", "Start", "Duration"], axis = 1)
columns  = pd.get_dummies(columns).columns

In [11]:
for i in range(10):
    print(record[i])

MannwhitneyuResult(statistic=array([220. ,  59. , 224. , 155.5, 150. ,  40. ,  30. , 345. , 332.5,
       339.5, 213.5, 254.5, 169.5, 257.5, 331. , 335.5, 352. , 368.5,
        81.5,  92.5,  90.5, 221.5, 180. , 300. , 260. ,  70. , 180. ,
       170. , 230. ]), pvalue=array([1.52078081e-01, 1.34633406e-04, 5.15695259e-01, 2.28432286e-01,
       1.82558606e-02, 3.41417358e-07, 7.91774464e-08, 3.79464065e-05,
       1.68711827e-04, 7.54328489e-05, 7.14864748e-01, 1.39805144e-01,
       4.02064413e-01, 1.09867548e-01, 1.97743384e-04, 1.19404869e-04,
       1.60932711e-05, 1.66753479e-06, 6.88615576e-04, 2.28993221e-03,
       1.88777259e-03, 2.64012287e-01, 4.35023231e-01, 1.69971695e-03,
       1.91871784e-02, 4.81701592e-05, 5.23879020e-01, 3.33308667e-01,
       3.33308667e-01]))
MannwhitneyuResult(statistic=array([198. ,  61.5, 231.5, 167. , 160. ,  50. ,  30. , 352. , 342. ,
       348.5, 210. , 300. , 157.5, 204.5, 336.5, 338. , 364. , 376. ,
        81. ,  67. ,  70. , 242.5, 200. 

In [12]:
df_stats = pd.DataFrame()

In [13]:
for i in range(10):
    new_stats = pd.DataFrame(data = record[i], columns = columns)
    df_stats = pd.concat([df_stats, new_stats], axis = 0)

We decide that a confidence level of 95% is required to reject the null hypothesis in favor of the alternative that the distributions are different.

In [14]:
df_stats

,Sampling rate(Hz),Age,Height (cm),Weight (kg),History of MI,History of CABG,History of PCI,AOD (mm),LAD (mm),LVDs,...,Aortic valve area Mean gradient(mmHg),Aortic valve area Peak velocity (m/sec),Mitral valve area (cm2),Moderate or greater MS,Moderate or greater MR,Moderate or greater AR,Moderate or greater AS,moderate or greater TR,Gender_F,Gender_M
0,220.000000,59.000000,224.000000,155.500000,150.000000,4.000000e+01,3.000000e+01,3.450000e+02,332.500000,3.395000e+02,...,92.500000,90.500000,221.500000,180.000000,300.000000,260.000000,7.000000e+01,180.000000,170.000000,230.000000
1,0.152078,0.000135,0.515695,0.228432,0.018256,3.414174e-07,7.917745e-08,3.794641e-05,0.000169,7.543285e-05,...,0.002290,0.001888,0.264012,0.435023,0.001700,0.019187,4.817016e-05,0.523879,0.333309,0.333309
0,198.000000,61.500000,231.500000,167.000000,160.000000,5.000000e+01,3.000000e+01,3.520000e+02,342.000000,3.485000e+02,...,67.000000,70.000000,242.500000,200.000000,280.000000,300.000000,6.000000e+01,190.000000,190.000000,210.000000
1,0.917246,0.000176,0.393104,0.371729,0.037373,1.315785e-06,7.917745e-08,1.339336e-05,0.000048,2.120914e-05,...,0.000161,0.000224,0.064232,1.000000,0.012053,0.000657,1.115292e-05,0.752137,0.738883,0.738883
0,198.000000,61.000000,210.000000,124.500000,170.000000,5.000000e+01,2.000000e+01,3.590000e+02,343.500000,3.420000e+02,...,107.000000,99.000000,260.000000,230.000000,240.000000,260.000000,6.000000e+01,170.000000,190.000000,210.000000
1,0.917246,0.000167,0.786491,0.040991,0.075363,1.315785e-06,1.615593e-08,4.986160e-06,0.000040,4.789635e-05,...,0.009753,0.004726,0.008987,0.262018,0.190430,0.019187,1.115292e-05,0.317311,0.752137,0.752137
0,200.500000,20.000000,202.500000,104.000000,170.000000,4.000000e+01,2.000000e+01,3.720000e+02,364.000000,3.725000e+02,...,108.000000,87.000000,232.000000,190.000000,300.000000,260.000000,6.000000e+01,220.000000,200.000000,200.000000
1,0.971437,0.000001,0.946023,0.009359,0.075363,3.414174e-07,1.615593e-08,1.035737e-06,0.000003,9.798891e-07,...,0.010971,0.001769,0.131922,0.681153,0.001793,0.019187,1.115292e-05,0.512691,1.000000,1.000000
0,199.500000,42.000000,229.000000,132.500000,170.000000,4.000000e+01,2.000000e+01,3.700000e+02,361.000000,3.530000e+02,...,58.000000,59.500000,270.000000,230.000000,270.000000,280.000000,4.000000e+01,220.000000,180.000000,220.000000
1,0.971437,0.000019,0.432299,0.067533,0.075363,3.414174e-07,1.615593e-08,1.369289e-06,0.000005,1.411973e-05,...,0.000064,0.000067,0.004250,0.217698,0.027052,0.003921,5.853215e-07,0.532299,0.512691,0.512691


In [15]:
df_stats.reset_index().drop(["index"], axis = 1)

,Sampling rate(Hz),Age,Height (cm),Weight (kg),History of MI,History of CABG,History of PCI,AOD (mm),LAD (mm),LVDs,...,Aortic valve area Mean gradient(mmHg),Aortic valve area Peak velocity (m/sec),Mitral valve area (cm2),Moderate or greater MS,Moderate or greater MR,Moderate or greater AR,Moderate or greater AS,moderate or greater TR,Gender_F,Gender_M
0,220.000000,59.000000,224.000000,155.500000,150.000000,4.000000e+01,3.000000e+01,3.450000e+02,332.500000,3.395000e+02,...,92.500000,90.500000,221.500000,180.000000,300.000000,260.000000,7.000000e+01,180.000000,170.000000,230.000000
1,0.152078,0.000135,0.515695,0.228432,0.018256,3.414174e-07,7.917745e-08,3.794641e-05,0.000169,7.543285e-05,...,0.002290,0.001888,0.264012,0.435023,0.001700,0.019187,4.817016e-05,0.523879,0.333309,0.333309
2,198.000000,61.500000,231.500000,167.000000,160.000000,5.000000e+01,3.000000e+01,3.520000e+02,342.000000,3.485000e+02,...,67.000000,70.000000,242.500000,200.000000,280.000000,300.000000,6.000000e+01,190.000000,190.000000,210.000000
3,0.917246,0.000176,0.393104,0.371729,0.037373,1.315785e-06,7.917745e-08,1.339336e-05,0.000048,2.120914e-05,...,0.000161,0.000224,0.064232,1.000000,0.012053,0.000657,1.115292e-05,0.752137,0.738883,0.738883
4,198.000000,61.000000,210.000000,124.500000,170.000000,5.000000e+01,2.000000e+01,3.590000e+02,343.500000,3.420000e+02,...,107.000000,99.000000,260.000000,230.000000,240.000000,260.000000,6.000000e+01,170.000000,190.000000,210.000000
5,0.917246,0.000167,0.786491,0.040991,0.075363,1.315785e-06,1.615593e-08,4.986160e-06,0.000040,4.789635e-05,...,0.009753,0.004726,0.008987,0.262018,0.190430,0.019187,1.115292e-05,0.317311,0.752137,0.752137
6,200.500000,20.000000,202.500000,104.000000,170.000000,4.000000e+01,2.000000e+01,3.720000e+02,364.000000,3.725000e+02,...,108.000000,87.000000,232.000000,190.000000,300.000000,260.000000,6.000000e+01,220.000000,200.000000,200.000000
7,0.971437,0.000001,0.946023,0.009359,0.075363,3.414174e-07,1.615593e-08,1.035737e-06,0.000003,9.798891e-07,...,0.010971,0.001769,0.131922,0.681153,0.001793,0.019187,1.115292e-05,0.512691,1.000000,1.000000
8,199.500000,42.000000,229.000000,132.500000,170.000000,4.000000e+01,2.000000e+01,3.700000e+02,361.000000,3.530000e+02,...,58.000000,59.500000,270.000000,230.000000,270.000000,280.000000,4.000000e+01,220.000000,180.000000,220.000000
9,0.971437,0.000019,0.432299,0.067533,0.075363,3.414174e-07,1.615593e-08,1.369289e-06,0.000005,1.411973e-05,...,0.000064,0.000067,0.004250,0.217698,0.027052,0.003921,5.853215e-07,0.532299,0.512691,0.512691


In [16]:
df_p = df_stats.drop([i for i in df_stats.index if (df_stats.index[i] %2 ==0)], axis=0)

In [17]:
df_p.reset_index(inplace =True)

In [18]:
df_p.style.apply(lambda x: ["background: yellow" if v < 0.05 else "" for v in x], axis = 1)

,index,Sampling rate(Hz),Age,Height (cm),Weight (kg),History of MI,History of CABG,History of PCI,AOD (mm),LAD (mm),LVDs,Ejection fraction (%),Left ventricular end diastolic dimension (mm),IVS (mm),LVPW (mm),RAD,RVDd,FS,TAPSE,Aortic valve area (cm2),Aortic valve area Mean gradient(mmHg),Aortic valve area Peak velocity (m/sec),Mitral valve area (cm2),Moderate or greater MS,Moderate or greater MR,Moderate or greater AR,Moderate or greater AS,moderate or greater TR,Gender_F,Gender_M
0,1,0.152078,0.000135,0.515695,0.228432,0.018256,0.000000,0.000000,0.000038,0.000169,0.000075,0.714865,0.139805,0.402064,0.109868,0.000198,0.000119,0.000016,0.000002,0.000689,0.002290,0.001888,0.264012,0.435023,0.001700,0.019187,0.000048,0.523879,0.333309,0.333309
1,1,0.917246,0.000176,0.393104,0.371729,0.037373,0.000001,0.000000,0.000013,0.000048,0.000021,0.786716,0.006776,0.244006,0.899782,0.000092,0.000078,0.000003,0.000000,0.000653,0.000161,0.000224,0.064232,1.000000,0.012053,0.000657,0.000011,0.752137,0.738883,0.738883
2,1,0.917246,0.000167,0.786491,0.040991,0.075363,0.000001,0.000000,0.000005,0.000040,0.000048,0.579005,0.151018,0.260843,0.083019,0.000065,0.000077,0.000004,0.000005,0.000835,0.009753,0.004726,0.008987,0.262018,0.190430,0.019187,0.000011,0.317311,0.752137,0.752137
3,1,0.971437,0.000001,0.946023,0.009359,0.075363,0.000000,0.000000,0.000001,0.000003,0.000001,0.357457,0.039569,0.048610,0.374127,0.000012,0.000006,0.000010,0.000010,0.000043,0.010971,0.001769,0.131922,0.681153,0.001793,0.019187,0.000011,0.512691,1.000000,1.000000
4,1,0.971437,0.000019,0.432299,0.067533,0.075363,0.000000,0.000000,0.000001,0.000005,0.000014,0.588226,0.063407,0.122138,0.219357,0.000016,0.000019,0.000001,0.000001,0.000342,0.000064,0.000067,0.004250,0.217698,0.027052,0.003921,0.000001,0.532299,0.512691,0.512691
5,1,0.958567,0.000027,0.597519,0.017163,0.037373,0.000001,0.000000,0.000006,0.000021,0.000048,0.490192,0.042270,0.290110,0.325586,0.000040,0.000047,0.000004,0.000027,0.005026,0.026986,0.012195,0.008987,0.262018,0.010787,0.008867,0.000131,0.333309,1.000000,1.000000
6,1,0.958567,0.000020,0.913720,0.015990,0.018256,0.000001,0.000000,0.000025,0.000039,0.000026,0.766014,0.022076,0.059586,0.802331,0.000051,0.000057,0.000003,0.000002,0.000024,0.000584,0.000307,0.264012,0.435023,0.004477,0.003921,0.000011,0.530236,1.000000,1.000000
7,1,nan,0.000005,0.378615,0.143814,0.037373,0.000000,0.000000,0.000001,0.000002,0.000000,0.490171,0.006652,0.233874,0.771574,0.000003,0.000003,0.000002,0.000001,0.000002,0.000050,0.000009,0.064265,1.000000,0.012053,0.008867,0.000000,0.752137,1.000000,1.000000
8,1,0.741767,0.000034,0.734921,0.076265,0.037373,0.000005,0.000000,0.000272,0.000292,0.000186,0.684799,0.284422,0.010695,0.707304,0.000406,0.000344,0.000088,0.000123,0.000735,0.002652,0.005614,0.064232,1.000000,0.055859,0.080300,0.000513,0.523879,0.530236,0.530236
9,1,0.971437,0.000096,0.464735,0.041029,0.037373,0.000001,0.000000,0.000024,0.000021,0.000050,0.101632,0.560207,0.649198,0.034142,0.000050,0.000060,0.000003,0.000010,0.000313,0.003982,0.002870,0.030397,1.000000,0.012490,0.003921,0.000048,0.343400,0.523879,0.523879
